# Dynamic Programming For Cutting Problems

In [14]:
from collections import defaultdict
import time

In [27]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i + 1] = p
print(price)
assert price[1] == 1

defaultdict(<class 'int'>, {1: 1, 2: 5, 3: 8, 4: 9, 5: 10, 6: 17, 7: 17, 8: 20, 9: 24, 10: 30})


In [16]:
def func_1(n):
    for i in range(n):
        print(n)

In [17]:
def call_time(func_1, arg):
    start = time.time()
    func_1(arg)
    print('used time:{}'.format(time.time() - start))

In [18]:
from functools import wraps

In [19]:
function_called_time = defaultdict(int)

def get_call_time(func):
    @wraps(func)
    def _inner(arg):
        global function_called_time
        function_called_time[func.__name__] += 1
        result = func(arg)
        print('function called time is :{}'.format(function_called_time[func.__name__]))
        return result
    return _inner

In [20]:
call_time(func_1, 10)

10
10
10
10
10
10
10
10
10
10
used time:0.005075931549072266


In [21]:
func_1 = get_call_time(func_1)

In [22]:
func_1(10)

10
10
10
10
10
10
10
10
10
10
function called time is :1


In [25]:
@get_call_time
def func_1(n):
    """
    @param n: is the number of customers
    @return int: the customers value point
    """

    for i in range(n):
        print(n)
    return 0

In [26]:
help(func_1)

Help on function func_1 in module __main__:

func_1(n)
    @param n: is the number of customers
    @return int: the customers value point



In [28]:
func_1(10)

10
10
10
10
10
10
10
10
10
10
function called time is :2


0

In [33]:
@get_call_time
def func_slow(n):
    for i in range(n):
        time.sleep(0.2)
        print(n)

In [36]:
func_slow(5)

5
5
5
5
5
function called time is :4


In [39]:
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(n):
        if n in cache: result = cache[n]
        else:
            result = func(n)
            cache[n] = result
        return result
    return _wrap

In [45]:
solution = {}

In [46]:
@memo
def r(n):
    max_price, split_point = max(
    [(price[n], 0)] + [(r(i) + r(n - i), i) for i in range(1, n)], key= lambda x: x[0])
    solution[n] = (split_point, n - split_point)
    
    return max_price

In [47]:
r(231)

691

In [48]:
solution[18]

(2, 16)

In [49]:
def not_cut(split): return split == 0

In [52]:
def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    
    if not_cut(left): return [right]
    
    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)

In [53]:
parse_solution(19, solution)

[3, 6, 10]

# Edit Distance

In [61]:
solution = {}

In [62]:
from functools import lru_cache

In [73]:
@lru_cache(maxsize = 2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidate = [
        (edit_distance(string1[:-1],string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2))
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {}=>{}'.format(tail_s1, tail_s2))
    candidate.append(both_forward)
    min_distance, operation = min(candidate, key = lambda x: x[0])
    solution[(string1, string2)] = operation
    
    return min_distance

In [75]:
edit_distance('ABCDE', 'ABCCEF')

2

In [76]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D=>C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}